# SELF-BLEU

In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import copy
import json
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu

#START: COPIED FROM https://blog.paperspace.com/automated-metrics-for-evaluating-generated-text/
def get_bleu_score(sentence, remaining_sentences):
    lst = []
    for i in remaining_sentences:
        bleu = sentence_bleu([sentence.split()], i.split())
        lst.append(bleu)
    return lst

def calculate_selfBleu(sentences):
    '''
    sentences - list of sentences generated by NLG system
    '''
    bleu_scores = []
	
    for i in sentences:
        sentences_copy = copy.deepcopy(sentences)
        remaining_sentences = sentences_copy.remove(i)
        bleu = get_bleu_score(i,sentences_copy)
        bleu_scores.append(bleu)

    return np.mean(bleu_scores)
#END: COPIED FROM https://blog.paperspace.com/automated-metrics-for-evaluating-generated-text/

sentences = []
with open("/Users/angela/Documents/NLPResults/Datasets/BT_only.json") as f:
    dataset = json.load(f)

emotion = "trust"
for k in dataset:
    emo = 0
    for annotation in dataset[k]['Annotations']:
        for hit in dataset[k]['Annotations'][annotation]:
            if hit['Emotion'] == emotion:
                emo = 1
    if emo == 1:
        sentences.append(dataset[k]['Reddit Post']) #retrieve backtranslated reddit post

print(sentences)
calculate_selfBleu(sentences)

[nltk_data] Downloading package punkt to /Users/angela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["Hi everyone, I hope you can help clarify some confusion. I'm still reading headlines about how the booster is ineffective against the omicron and this makes me think that getting a booster is a kind of meaningless risk - why risk the side effects of the booster when it won't even help me? But I also think, are the other older variants (delta, original Covid) still in circulation? Maybe I should be vaccinated to protect myself against them? I don't know. I've already taken the first two doses of the vaccine <ORG>, and I took my second dose again in July 2021. I had little to any side effect. I've had enough to continue working. But I've always been afraid to receive the vaccines due to the possible side effects. I'm a 30-year-old male in normal health and normal weight.", 'Good news from my country at last! First of all, it seems that support is growing everywhere to lift all the remaining restrictions! Many leaders in health and politics, including the hospital directors and chief ep

0.0008107826373244391

# BLEU SCORE - Compare Backtranslation w/ Original Post


In [ ]:
with open("/Users/angela/Documents/BT_only_new.json") as f:
    dataset = json.load(f)
bleu_scores = []
emotion = "resilience"
c = 0
for k in dataset:
    emo = 0
    original = None
    for annotation in dataset[k]['Annotations']:
        for hit in dataset[k]['Annotations'][annotation]:
            original = hit["Reddit Post Original"]
            if hit['Emotion'] == emotion:
                emo = 1
    if emo == 1:
        bleu = sentence_bleu([original.split()], dataset[k]['Reddit Post'][0].split())
        bleu_scores.append(bleu)
print(np.average(bleu_scores))

# Perplexity

In [ ]:
!pip install lmppl

In [ ]:
import lmppl
import json
import numpy as np

with open('/Users/angela/Documents/NLPResults/Backtranslation/Disgust/summaries_output_for_bertcombined_backtranslation.txt_disgust.txt', 'r') as file:
    lines = file.readlines()
    our_summaries = json.loads(lines[-4]) #retrieve list of generated summaries
print(our_summaries)

#START: COPIED FROM https://github.com/asahi417/lmppl
scorer = lmppl.LM('gpt2')
ppl = scorer.get_perplexity(our_summaries)
#END: COPIED FROM https://github.com/asahi417/lmppl

print(list(zip(our_summaries, ppl)))
print(np.average(ppl))

# SummaCConv

SummmaCConv scores for target, annotated summaries

In [ ]:
import numpy 
from summac.model_summac import SummaCConv
import json

original_docs = []
summaries = []
emotion = "trust"

with open(original_docs_fname, 'r') as file:
    dataset = json.load(file)
    summary = None
    for k in dataset:
        emo = 0
        for annotation in dataset[k]['Annotations']:
            for hit in dataset[k]['Annotations'][annotation]:
                if hit['Emotion'] == emotion:
                    emo = 1
                    summary = hit['Abstractive']
        if emo == 1:
            original_docs.append(dataset[k]['Reddit Post']) #retrieve original reddit post
            summaries.append(summary) #retrieve target, annotated summaries

scores = []

#START: COPIED FROM https://github.com/tingofurro/summac
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")
for i, item in enumerate(original_docs):
    score_conv1 = model_conv.score([original_docs[i]], [summaries[i]])
    #END: COPIED FROM https://github.com/tingofurro/summac
    scores.append(score_conv1["scores"][0])
    print("SummacConv score: %.3f" % (score_conv1["scores"][0]))

print(numpy.average(scores))

SummmaCConv scores for generated summaries

In [ ]:
import numpy 
from summac.model_summac import SummaCConv
import json

original_docs = []
generated_summaries = []
emotion = "trust"

with open(summaries_fname) as f:
   lines = f.readlines()
    our_summaries = json.loads(lines[-4])
    for summ in our_summaries:
        generated_summaries.append(summ) #retrieve generated summaries 

with open(original_docs_fname, 'r') as file:
    dataset = json.load(file)
    summary = None
    for k in dataset:
        emo = 0
        for annotation in dataset[k]['Annotations']:
            for hit in dataset[k]['Annotations'][annotation]:
                if hit['Emotion'] == emotion:
                    emo = 1
                    summary = hit['Abstractive']
        if emo == 1:
            original_docs.append(dataset[k]['Reddit Post']) #retrieve original reddit post

scores = []

#START: COPIED FROM https://github.com/tingofurro/summac
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")
for i, item in enumerate(original_docs):
    score_conv1 = model_conv.score([original_docs[i]], [generated_summaries[i]])
    #END: COPIED FROM https://github.com/tingofurro/summac

    scores.append(score_conv1["scores"][0])
    print("SummacConv score: %.3f" % (score_conv1["scores"][0]))

print(numpy.average(scores))

# BERTScore

In [ ]:
from bert_score import score as bert_scr
import json 

f1_scores = []
i = 0
with open('/Users/angela/Documents/summaries_output_for_bertall_.txt_disgust.txt', 'r') as file:
    lines = file.readlines()
    for i in range(0, len(lines), 2):
        our_summaries = json.loads(lines[i]) #retrieve generated summaries
        target_summaries = json.loads(lines[i+1]) #retrieve target summaries 
        
        #START: COPIED FROM https://github.com/honglizhan/CovidET/tree/main
        _, _, F1 = bert_scr(our_summaries,
                        target_summaries,
                        model_type='microsoft/deberta-xlarge-mnli',
                        lang='en',
                        verbose=True)
        f1_scores.append(float(F1.mean()))
        #END: COPIED FROM https://github.com/honglizhan/CovidET/tree/main

print(f1_scores)

In [ ]:
path = '/Users/angela/Documents/all_.txt_disgust.json'
with open(path, 'r') as file:
    data = json.load(file) 
    for i, item in enumerate(data):
        data[item][3] = f1_scores[i] #load output file generated by detection_summarization.py, replace placeholder value w/ calculated BERTScore
with open(path, 'a') as file:
    json.dump(data, file)